In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1610140017389_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, r.email, r.review_length, \
case when (contains(r.observations, 'TST') )  then 1 else 0 end as TST \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR')\
and r.created_date > '2020-01-01' "


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(20)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                NAME  ... REVIEW_LENGTH TST
0   321715206                          Shutterfly  ...          92.0   0
1   321683397                          Shutterfly  ...         500.0   0
2   321705652                              Bealls  ...          39.0   0
3   321674298                              Bealls  ...          19.0   0
4   321687091                              Bealls  ...         439.0   0
5   321708926                              Bealls  ...          37.0   0
6   321713579                          Shutterfly  ...          76.0   0
7   321682036  International Delight - TERMINATED  ...         625.0   0
8   321670529                              Bealls  ...         116.0   0
9   321658073                              Bealls  ...          67.0   0
10  321665018                              Bealls  ...          74.0   0
11  321709008                              Bealls  ...         123.0   0
12  321709115                 Venus Fashion, Inc.  

In [6]:
reviews['HEADLINE_LENGTH']=reviews['REVIEW_HEADLINE'].apply(lambda x: len(x))

reviews['ALL_TEXT'] =  reviews['NAME']+ ' '+reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                           ALL_TEXT
0  321715206  ...  Shutterfly Beautiful cards... I was very happy...
1  321683397  ...  Shutterfly I would never again place any order...
2  321705652  ...  Bealls I have yet to receive this order, place...
3  321674298  ...         Bealls Did not receive Did not receive it.
4  321687091  ...  Bealls Gave as a gift I added the two products...

[5 rows x 9 columns]

In [7]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6537239 entries, 0 to 6537238
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   UGC_ID           int64  
 1   NAME             object 
 2   REVIEW_HEADLINE  object 
 3   REVIEW_COMMENTS  object 
 4   EMAIL            object 
 5   REVIEW_LENGTH    float64
 6   TST              int64  
 7   HEADLINE_LENGTH  int64  
 8   ALL_TEXT         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 448.9+ MB

In [123]:
#Not words like greatest, or tested on ..
#reviews['regex_otherwords'] = reviews['ALL_TEXT'].str.replace(r'(GREATEST|TESTED)', '', case=False)

#Contains words related to testing
reviews['regex_headline'] = reviews['REVIEW_HEADLINE'].str.contains(r'\b(TESTING|TEST|TESTED|TEST1|TESTT)\b', case=False)==True

reviews['regex_testoneword'] = reviews['ALL_TEXT'].str.contains(r'\b(TESTING|TEST)\b', case=False)==True

reviews['regex_thisisatest'] = reviews['ALL_TEXT'].str.contains(r'(THIS IS A TEST)\b', case=False)==True

#LOOK FOR WHERE TEST OCCURS MORE THAN ONCE
reviews['regex_testmultiwords'] = reviews['ALL_TEXT'].str.contains(r'\b(TESTING|TEST)\s?(TESTING|TEST){1,}\b', case=False)==True

#Email - just looking for "test" within email address returns tons of FP bc matches all sorts of words w/ test in them. 
#greatest@gmail.com
reviews['regex_email'] = reviews['EMAIL'].str.contains(r'\b(TESTING|TEST)\@', case=False)==True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [124]:
#<15 so don't get long reviews mentioning other tests not related to a test review but do get "test review" = 11 chars and "testing review" = 14 chars
reviews['regex_TST'] = 0
reviews.loc[((reviews['regex_testoneword']==True) & (reviews['REVIEW_LENGTH']<30)) | (reviews['regex_testmultiwords']==True)
            |((reviews['regex_headline']==True) & (reviews['HEADLINE_LENGTH']<15)) |  (reviews['regex_email']==True)
             | (reviews['regex_thisisatest']==True), 'regex_TST'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID        NAME  \
0  321715206  Shutterfly   
1  321683397  Shutterfly   
2  321705652  Bealls       
3  321674298  Bealls       
4  321687091  Bealls       

                                                REVIEW_HEADLINE  \
0  Beautiful cards...                                             
1  I would never again place any order with Shutterfly            
2  I have yet to receive this order, placed dec7/20 not happy!!   
3  Did not receive                                                
4  Gave as a gift                                                 

                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [126]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#reviews.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [127]:
FP = reviews[(reviews['TST'] == 0) & (reviews['regex_TST'] == 1)]
FN = reviews[(reviews['TST'] == 1) & (reviews['regex_TST'] == 0)]
TP = reviews[(reviews['TST'] == 1) & (reviews['regex_TST'] == 1)]
TN = reviews[(reviews['TST'] == 0) & (reviews['regex_TST'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
#Vast majority of these are not FP but missed being labeled
#Can see that by looking at review headline which is mostly "reviewHeadline", 
#comment which is "reviewComment" and email is "test@test.com" or #test@email.com"
FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

705

In [129]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1272

In [130]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5063

In [131]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6530199

In [132]:
FN.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           UGC_ID                               NAME  \
9108    321378619  Calvin Klein                        
14917   265653735  Big Lots                            
30248   273460394  Bridgestone Tire                    
31943   283530482  Bissell Homecare, Inc.              
34193   304880196  Dick Blick                          
42860   264679186  PR Demo Store                       
44463   298867636  SpeedoUSA.com                       
44464   298864367  SpeedoUSA.com                       
48611   268151826  Discount Ramps                      
48612   268152255  Discount Ramps                      
49171   279766222  Team Express                        
50707   264889289  Big Lots                            
50708   264889233  Big Lots                            
67043   298782136  Calvin Klein                        
67047   298824419  Calvin Klein                        
77663   284963941  Lascana                             
79817   320900219  Melaleuca US                 

In [134]:
#FPs on things like: DNA test, test on my spouse first, 
FP.head(100)
#test=FP[FP['regex_thisisatest']==True]
#test.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           UGC_ID                            NAME  \
233     321655428  EverlyWell                       
948     320411518  EverlyWell                       
22128   280574197  Sam Ash                          
30249   273440342  Bridgestone Tire                 
30253   273435100  Bridgestone Tire                 
42243   301184962  Dover Saddlery                   
52063   318501998  CompTIA                          
66569   298828119  MSC Industrial Supply            
97366   271876040  Ciex Testing                     
97682   271838878  Star Furniture Test              
114103  297095826  MSC Industrial Supply            
119576  302029464  EverlyWell                       
126768  308442398  EverlyWell                       
132382  269224446  Ulta                             
139258  284590536  GardenTech                       
144464  278344989  MSC Industrial Supply            
144652  278374273  Pampered Chef                    
144890  264344253  Dick Blick TEST            

In [17]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("PR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("regex_range_$", StringType(), True)\
,StructField("regex_range_dollarbuck", StringType(), True)\
,StructField("regex_shipping_$", StringType(), True)\
,StructField("regex_shipping_dollarbuck", StringType(), True)\
,StructField("regex_price", BooleanType(), True)\
,StructField("regex_PR", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : 'APP_SPARK_PROD',
  'sfPassword' : '>6txhcQ*7&^Qrq`j',
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","PR_FP_test1").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FN_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FN_test2")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FP_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.PR_FP_test2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…